In [ ]:
Libraries <- c('standR', 'SpatialExperiment', 'limma', 'ExperimentHub',
               'ggalluvial','optparse')
lapply(Libraries, require, character.only = TRUE)
rm(Libraries)


In [ ]:
# Choose whether to save local copies of images in the StandR working directory
exportFigs = FALSE

In [ ]:
setwd("/Users/upton6/Documents/Nanostring/projects/Dando/DSP_Output")


In [ ]:
countFile <- "countFile.tsv"
sampleAnnoFile <- "sampleAnnoFile.tsv"
featureAnnoFile <- "/Users/upton6/Documents/Nanostring/Projects/Dando/DSP_Output/featureAnnoFile.tsv"

In [ ]:
spe <- readGeoMx(countFile, sampleAnnoFile, 
                 featureAnnoFile = featureAnnoFile, rmNegProbe = FALSE)


In [ ]:
# Set the StandR working directory
setwd("/Users/upton6/Documents/Nanostring/projects/Dando/DSP_Output/StandR")

In [ ]:
colData(spe)$regions <- paste0(colData(spe)$Tissue,"_",colData(spe)$Region,"_",colData(spe)$Layer)


In [ ]:
plotSampleInfo(spe, column2plot = c("Tissue","Region","Layer"))

if (exportFigs){
    pdfName <- 'SampleInfo_spe.pdf'
    pdf( pdfName, width = 8 , height = 4 )
    plotSampleInfo(spe, column2plot = c("Tissue","Region","Layer"))
    dev.off()
}



In [ ]:
spe <- addPerROIQC(spe, rm_genes = TRUE)

plotGeneQC(spe, ordannots = "regions", col = regions, point_size = 2, bin_num = 30, top_n=90)

if (exportFigs){
    pdfName <- 'GeneQC_spe.pdf'
    pdf( pdfName, width = 8 , height = 4 )
    plotGeneQC(spe, ordannots = "regions", col = regions, point_size = 2, bin_num = 30, top_n=90)
    dev.off()
}

In [ ]:
plotROIQC(spe, y_threshold = 100000, col = colData(spe)$ScanLabel)

if (exportFigs){
    pdfName <- 'ROIQC_spe.pdf'
    pdf( pdfName, width = 8 , height = 4 )
    plotROIQC(spe, y_threshold = 1000000, col = SlideName)
    plotROIQC(spe, y_threshold = 100000, col = colData(spe)$ScanLabel)
    dev.off()
}

In [ ]:
spe <- spe[,rownames(colData(spe))[colData(spe)$lib_size > 100000]]


In [ ]:
plotRLExpr(spe, ordannots = "SegmentLabel", assay = 2, col = colData(spe)$SegmentLabel)


In [ ]:
plotRLExpr(spe, ordannots = "Tissue", assay = 2, col = colData(spe)$Tissue)


In [ ]:
plotRLExpr(spe, ordannots = "Region", assay = 2, col = colData(spe)$Region)


In [ ]:
plotRLExpr(spe, ordannots = "Layer", assay = 2, col = colData(spe)$Layer)


In [ ]:


if (exportFigs){
    pdfName <- 'RLE_spe.pdf'
    pdf( pdfName, width = 8 , height = 4 )
    plotRLExpr(spe, ordannots = "SegmentLabel", assay = 2, col = colData(spe)$SegmentLabel)
    plotRLExpr(spe, ordannots = "Tissue", assay = 2, col = colData(spe)$Tissue)
    plotRLExpr(spe, ordannots = "Region", assay = 2, col = colData(spe)$Region)
    plotRLExpr(spe, ordannots = "Layer", assay = 2, col = colData(spe)$Layer)
    dev.off()
}

In [ ]:
# colData(spe)$smlRegions <- paste0(colData(spe)$Clinical_outcome,"_",colData(spe)$Group)

# drawPCA(spe, assay = 2, col = Core, shape = smlRegions)
drawPCA(spe, assay = 2, col = Tissue, shape = Tissue)


In [ ]:
drawPCA(spe, assay = 2, col = Region, shape = Tissue)


In [ ]:
drawPCA(spe, assay = 2, col = Layer, shape = Tissue)


In [ ]:
if (exportFigs){
    pdfName <- 'PCAA_spe.pdf'
    pdf( pdfName, width = 20 , height = 20 )
    drawPCA(spe, assay = 2, col = Tissue, shape = Tissue)
    drawPCA(spe, assay = 2, col = Region, shape = Tissue)
    drawPCA(spe, assay = 2, col = Layer, shape = Tissue)
    dev.off()
}

In [ ]:
set.seed(100)
spe <- scater::runPCA(spe)
pca_results <- reducedDim(spe, "PCA")
plotScreePCA(spe, precomputed = pca_results)


if (exportFigs){
    pdfName <- 'PCAA_Scree_spe.pdf'
    pdf( pdfName, width = 8 , height = 4 )
    plotScreePCA(spe, precomputed = pca_results)
    dev.off()
}

In [ ]:
colData(spe)$biology <- paste0(colData(spe)$Tissue, "_", colData(spe)$Region, "_", colData(spe)$Layer)


In [ ]:
spe_tmm <- geomxNorm(spe, method = "TMM")


In [ ]:
spe <- findNCGs(spe, batch_name = "ScanLabel", top_n = 500)


In [ ]:
metadata(spe) |> names()


In [ ]:


# # Here we use k of 5 to perform RUV-4 normalization.

# # pdfName <- 'PCA_series_Core-Clinical_outcome-Time_point-Group_spe.pdf'
# # pdf( pdfName, width = 8 , height = 4 )

# for(i in seq(20)){
#   spe_ruv <- geomxBatchCorrection(spe, factors = "biology", 
#                                   NCGs = metadata(spe)$NCGs, k = i)
#   plotPairPCA(spe_ruv, assay = 2, n_dimension = 4, color = Tissue, title = paste0("k = ", i))
#   plotPairPCA(spe_ruv, assay = 2, n_dimension = 4, color = Region, title = paste0("k = ", i))
#   plotPairPCA(spe_ruv, assay = 2, n_dimension = 4, color = Layer, title = paste0("k = ", i))
  
# }
# # dev.off()




In [ ]:
spe_ruv2 <- geomxBatchCorrection(spe, factors = "biology", 
                                NCGs = metadata(spe)$NCGs, k = 2)
spe_ruv3 <- geomxBatchCorrection(spe, factors = "biology", 
                                 NCGs = metadata(spe)$NCGs, k = 3)
spe_ruv4 <- geomxBatchCorrection(spe, factors = "biology", 
                                 NCGs = metadata(spe)$NCGs, k = 4)
spe_ruv5 <- geomxBatchCorrection(spe, factors = "biology", 
                                NCGs = metadata(spe)$NCGs, k = 5)
spe_ruv6 <- geomxBatchCorrection(spe, factors = "biology", 
                                 NCGs = metadata(spe)$NCGs, k = 6)
spe_ruv7 <- geomxBatchCorrection(spe, factors = "biology", 
                                 NCGs = metadata(spe)$NCGs, k = 7)
spe_ruv8 <- geomxBatchCorrection(spe, factors = "biology", 
                                 NCGs = metadata(spe)$NCGs, k = 8)
spe_ruv9 <- geomxBatchCorrection(spe, factors = "biology", 
                                 NCGs = metadata(spe)$NCGs, k = 9)
spe_ruv10 <- geomxBatchCorrection(spe, factors = "biology", 
                                NCGs = metadata(spe)$NCGs, k = 10)
spe_ruv15 <- geomxBatchCorrection(spe, factors = "biology", 
                                NCGs = metadata(spe)$NCGs, k = 15)
# spe_ruv20 <- geomxBatchCorrection(spe, factors = "biology", 
#                                 NCGs = metadata(spe)$NCGs, k = 20)



In [ ]:
spe_ruv15

In [ ]:
spe_list <- list(spe, spe_ruv2)
plotClusterEvalStats(spe_list = spe_list,
                     bio_feature_name = "biology",
                     batch_feature_name = "Tissue",
                     data_names = c("Raw","RUV4-2"))


In [ ]:
spe_list <- list(spe, spe_ruv2,spe_ruv3,spe_ruv4,spe_ruv5,spe_ruv6,spe_ruv7,spe_ruv8,spe_ruv9,spe_ruv10,spe_ruv15)
plotClusterEvalStats(spe_list = spe_list,
                     bio_feature_name = "biology",
                     batch_feature_name = "Tissue",
                     data_names = c("Raw","RUV4-2","RUV4-3","RUV4-4","RUV4-5","RUV4-6","RUV4-7","RUV4-8","RUV4-9","RUV410","RUV415"))


In [ ]:




# pdfName <- 'plotClusterEvalStats_spe_ruv.pdf'
# # ToDo: Increase number of shapes in plot. change colours to colour by KO or diet
# pdf( pdfName, width = 20 , height = 10 )



spe_list <- list(spe, spe_ruv2,spe_ruv3,spe_ruv4,spe_ruv5,spe_ruv6,spe_ruv7,spe_ruv8,spe_ruv9,spe_ruv10,spe_ruv15,spe_ruv20)
plotClusterEvalStats(spe_list = spe_list,
                     bio_feature_name = "Clinical_outcome",
                     batch_feature_name = "ScanLabel",
                     data_names = c("Raw","RUV4-2","RUV4-3","RUV4-4","RUV4-5","RUV4-6","RUV4-7","RUV4-8","RUV4-9","RUV410","RUV415","RUV420"))

# spe_list <- list(spe, spe_ruv2,spe_ruv3,spe_ruv4,spe_ruv5,spe_ruv6,spe_ruv7,spe_ruv8,spe_ruv9,spe_ruv10,spe_ruv15)
# plotClusterEvalStats(spe_list = spe_list,
#                      bio_feature_name = "Core",
#                      batch_feature_name = "ScanLabel",
#                      data_names = c("Raw","RUV4-2","RUV4-3","RUV4-4","RUV4-5","RUV4-6","RUV4-7","RUV4-8","RUV4-9","RUV410","RUV415","RUV420"))

spe_list <- list(spe, spe_ruv2,spe_ruv3,spe_ruv4,spe_ruv5,spe_ruv6,spe_ruv7,spe_ruv8,spe_ruv9,spe_ruv10,spe_ruv15,spe_ruv20)
plotClusterEvalStats(spe_list = spe_list,
                     bio_feature_name = "Time_point",
                     batch_feature_name = "ScanLabel",
                     data_names = c("Raw","RUV4-2","RUV4-3","RUV4-4","RUV4-5","RUV4-6","RUV4-7","RUV4-8","RUV4-9","RUV410","RUV415","RUV420"))

spe_list <- list(spe, spe_ruv2,spe_ruv3,spe_ruv4,spe_ruv5,spe_ruv6,spe_ruv7,spe_ruv8,spe_ruv9,spe_ruv10,spe_ruv15,spe_ruv20)
plotClusterEvalStats(spe_list = spe_list,
                     bio_feature_name = "Group",
                     batch_feature_name = "ScanLabel",
                     data_names = c("Raw","RUV4-2","RUV4-3","RUV4-4","RUV4-5","RUV4-6","RUV4-7","RUV4-8","RUV4-9","RUV410","RUV415","RUV420"))

dev.off()



In [ ]:

spe_ruv <- geomxBatchCorrection(spe, factors = "biology", 
                                NCGs = metadata(spe)$NCGs, k = 5)


In [ ]:
plotPairPCA(spe, assay = 2, color = Tissue, shape = Tissue, title = "RUV4")


In [ ]:
plotPairPCA(spe_ruv, assay = 2, color = Tissue, shape = Tissue, title = "RUV4")


In [ ]:
plotPairPCA(spe, assay = 2, color = Region, shape = Tissue, title = "RUV4")


In [ ]:
plotPairPCA(spe_ruv, assay = 2, color = Region, shape = Tissue, title = "RUV4")


In [ ]:
plotPairPCA(spe, assay = 2, color = Layer, shape = Tissue, title = "RUV4")


In [ ]:
plotPairPCA(spe_ruv, assay = 2, color = Layer, shape = Tissue, title = "RUV4")


In [ ]:

# # 
# # Moreover, we can also have a look at the RLE plots of the normalized count.
# # 
# pdfName <- 'RLE_spe_ruv.pdf'
# pdf( pdfName, width = 8 , height = 4 )
# plotRLExpr(spe_ruv, assay = 2, color = SegmentLabel) + ggtitle("RUV4")
# # plotRLExpr(spe_ruv, assay = 2, color = Core) + ggtitle("RUV4")
# plotRLExpr(spe_ruv, assay = 2, color = Clinical_outcome) + ggtitle("RUV4")
# plotRLExpr(spe_ruv, assay = 2, color = Time_point) + ggtitle("RUV4")
# plotRLExpr(spe_ruv, assay = 2, color = Group) + ggtitle("RUV4")
# dev.off()
# # 
# # 
# # **For more detailed analysis pipeline and usage of the standR package, please see https://github.com/DavisLaboratory/GeoMXAnalysisWorkflow**
# # 


In [ ]:
plotRLExpr(spe_ruv, assay = 2, color = Tissue) + ggtitle("RUV4")


In [ ]:
plotRLExpr(spe_ruv, assay = 2, color = Region) + ggtitle("RUV4")


In [ ]:
plotRLExpr(spe_ruv, assay = 2, color = Layer) + ggtitle("RUV4")


In [ ]:

# Write RUV corrected results to file
spe_RUV_out <- assay(spe_ruv, i=2)
spe_RUV_out <- 2**spe_RUV_out

write.table(spe_RUV_out, file = "RUV_corrected_values.csv", sep = ",", row.names = TRUE, col.names = TRUE)



# # SessionInfo
sessionInfo()
